In [1]:
import pandas as pd
import numpy as np
import re

# Load data processing

In [3]:
# Read original hourly load
region = pd.read_csv('Processed Data/Reduced_Area_Region.csv')
load = pd.read_csv('Public Data/Hourly Profiles in CSV Format/Hourly Profile_Load.csv')

In [3]:
# reduce column names to Area Names
def extract_name(name):
    match = re.search(r'_(\w+)_', name)
    return match.group(1) if match else name

load.columns = [extract_name(name) for name in load.columns]

print(load.columns)

Index(['Index', 'AESO', 'AVA', 'AZPS', 'BANC', 'BCHA', 'BPAT', 'CFE', 'CHPD',
       'CIPB', 'CIPV', 'CISC', 'CISD', 'DOPD', 'EPE', 'GCPD', 'IID', 'IPFE',
       'IPMV', 'IPTV', 'LDWP', 'NEVP', 'NWMT', 'PACW', 'PAID', 'PAUT', 'PAWY',
       'PGE', 'PNM', 'PSCO', 'PSEI', 'SCL', 'SPPC', 'SRP', 'TEPC', 'TH_Malin',
       'TH_Mead', 'TH_PV', 'TIDC', 'TPWR', 'VEA', 'WACM', 'WALC', 'WAUW',
       'Unnamed: 44'],
      dtype='object')


In [4]:
# filter load data and keep 8784 hours (rows) and drop last empty column
load = load.loc[1:8784]
load = load.drop(load.columns[-1], axis=1)

# set column index as index
load = load.set_index(load.columns[0])

# load data to numeric, originally string
load = load.apply(pd.to_numeric)

In [5]:
# Create an empty DataFrame for the results
load_reduced = pd.DataFrame()

# Loop over the unique values in df_map
for value in region['Zone'].unique():
    # Get the columns that are mapped to this value
    cols = region.loc[region['Zone'] == value, 'Area Name']
    
    # Sum these columns in df and add the result to df_result
    load_reduced[value] = load[cols].sum(axis=1)

In [6]:
sorted_columns = sorted(load_reduced.columns, key=lambda x: int(x))
load_reduced = load_reduced[sorted_columns]


In [7]:
load_reduced.to_csv('Processed Data/Load.csv')

# Thermal generator processing

In [8]:
region = pd.read_csv('Processed Data/Reduced_Area_Region.csv')
genlist = pd.read_csv('Public Data/GeneratorList.csv',header=2, encoding='ISO-8859-1', usecols=['Name','SubType','Area Name','InitialDispatch(MW)'])
thermalgenlist = pd.read_csv('Public Data/Thermal_General_Info.csv',header=1, encoding='ISO-8859-1', usecols=['GeneratorName','MustRun','Fuel Name','MinimumDownTime(hr)','MinimumUpTime(hr)','RampUp Rate(MW/minute)','RampDn Rate(MW/minute)','Startup Cost Fixed($)','Startup Fuel Name','StartFuel(MMBTu)','VOM Cost'])
thermalgencurve = pd.read_csv('Public Data/Thermal_IOCurve_Info.csv',header=1, encoding='ISO-8859-1',usecols=['Generator Name','If Use Generic IO Curve','Generic IO Curve Name','IONumBlock','IOMaxCap(MW)','IOMinCap(MW)','MinInput(MMBTu)','IncCap2(MW)','IncHR2(MMBTu/MWh)','IncCap3(MW)','IncHR3(MMBTu/MWh)','IncCap4(MW)','IncHR4(MMBTu/MWh)','IncCap5(MW)','IncHR5(MMBTu/MWh)','IncCap6(MW)','IncHR6(MMBTu/MWh)'])
fuelcost = pd.read_csv('Public Data/FuelCost_Monthly.csv',header=1, encoding='ISO-8859-1')
thermalgencurve = thermalgencurve[~thermalgencurve['If Use Generic IO Curve']]


In [9]:
# attach zone info to genlist
genlist['Zone'] = genlist['Area Name'].map(region.set_index('Area Name')['Zone'])

In [10]:
# calculate average fuel cost and reduce the dataframe to two columns ['Fuel Name','Fuel Cost($/MMBTu)']
fuelcost['Fuel Cost($/MMBTu)']=fuelcost.iloc[:, 3:15].mean(axis=1)
fuelcost = fuelcost.drop(fuelcost.columns[1:15], axis=1)

In [11]:
thermalgen = pd.merge(thermalgencurve, genlist, left_on='Generator Name', right_on='Name', how='inner')
thermalgen = pd.merge(thermalgen, thermalgenlist, left_on='Generator Name', right_on='GeneratorName', how='inner')

In [12]:
thermalgen = pd.merge(thermalgen, fuelcost, left_on='Fuel Name', right_on='Fuel Name', how='inner')
thermalgen = pd.merge(thermalgen, fuelcost, left_on='Startup Fuel Name', right_on='Fuel Name', how='inner', suffixes=(None, '_Start'))

In [13]:
# cost calculation base on fuel prices
thermalgen['Start Up Cost($)'] = thermalgen['Startup Cost Fixed($)'] + thermalgen['StartFuel(MMBTu)'] * thermalgen['Fuel Cost($/MMBTu)_Start']
thermalgen['NoLoadCost($)'] = thermalgen['MinInput(MMBTu)'] * thermalgen['Fuel Cost($/MMBTu)']
thermalgen['IncCost2($/MW)'] = thermalgen['IncHR2(MMBTu/MWh)'] * thermalgen['Fuel Cost($/MMBTu)']
thermalgen['IncCost3($/MW)'] = thermalgen['IncHR3(MMBTu/MWh)'] * thermalgen['Fuel Cost($/MMBTu)']
thermalgen['IncCost4($/MW)'] = thermalgen['IncHR4(MMBTu/MWh)'] * thermalgen['Fuel Cost($/MMBTu)']
thermalgen['IncCost5($/MW)'] = thermalgen['IncHR5(MMBTu/MWh)'] * thermalgen['Fuel Cost($/MMBTu)']
thermalgen['IncCost6($/MW)'] = thermalgen['IncHR6(MMBTu/MWh)'] * thermalgen['Fuel Cost($/MMBTu)']

In [14]:
thermalgen.to_csv('Processed Data/ThermalGen_Full.csv')

In [15]:
columns_to_drop = ['If Use Generic IO Curve', 'Generic IO Curve Name','IONumBlock', 'MinInput(MMBTu)',
                   'IncHR2(MMBTu/MWh)', 'IncHR3(MMBTu/MWh)', 'IncHR4(MMBTu/MWh)', 'IncHR5(MMBTu/MWh)','IncHR6(MMBTu/MWh)',
                  'Name', 'GeneratorName', 'Fuel Name', 'Startup Cost Fixed($)', 'Startup Fuel Name', 'StartFuel(MMBTu)',
                  'Fuel Cost($/MMBTu)', 'Fuel Name_Start', 'Fuel Cost($/MMBTu)_Start']
thermalgen = thermalgen.drop(columns=columns_to_drop)

In [16]:
thermalgen.iloc[0]

Generator Name            Ox_Mountain7
IOMaxCap(MW)                       1.9
IOMinCap(MW)                      0.57
IncCap2(MW)                       1.33
IncCap3(MW)                        0.0
IncCap4(MW)                        0.0
IncCap5(MW)                        0.0
IncCap6(MW)                        0.0
SubType                        Bio-ICE
Area Name                         CIPB
InitialDispatch(MW)               1.55
Zone                                 1
MustRun                           True
MinimumDownTime(hr)                  6
MinimumUpTime(hr)                    6
RampUp Rate(MW/minute)             4.0
RampDn Rate(MW/minute)             4.0
VOM Cost                      2.027986
Start Up Cost($)             11.155225
NoLoadCost2($)               16.432495
IncCost2($/MW)               28.830608
IncCost3($/MW)                     0.0
IncCost4($/MW)                     0.0
IncCost5($/MW)                     0.0
IncCost6($/MW)                     0.0
Name: 0, dtype: object

In [17]:
mask = thermalgen['InitialDispatch(MW)'] != 0
thermalgen.loc[mask, 'InitialDispatch(MW)'] = thermalgen.loc[mask, 'InitialDispatch(MW)'].clip(lower=thermalgen.loc[mask, 'IOMinCap(MW)'], upper=thermalgen.loc[mask, 'IOMaxCap(MW)'])

In [18]:
thermalgen['Initial Status'] = (thermalgen['InitialDispatch(MW)'] != 0).astype(int)

In [19]:
thermalgen.to_csv('Processed Data/ThermalGen.csv')

# Thermal generator map

In [20]:
thermalgenzone = pd.read_csv('Processed Data/ThermalGen.csv', usecols=['Generator Name','Zone'])

In [21]:
thermalgenmap = pd.get_dummies(thermalgenzone["Zone"])
thermalgenmap.to_csv('Processed Data/ThermalGenMap.csv')

# Wind  processing

In [22]:
genlist = pd.read_csv('Public Data/GeneratorList.csv',header=2, encoding='ISO-8859-1', usecols=['Name','SubType','Area Name','InitialDispatch(MW)'])
hourlygenlist = pd.read_csv('Public Data/HourlyResource_General.csv',header=2, encoding='ISO-8859-1')

In [23]:
# attach zone info to genlist
genlist['Zone'] = genlist['Area Name'].map(region.set_index('Area Name')['Zone'])
windlist = hourlygenlist[hourlygenlist['Type'] == 'Wind']
windlist = pd.merge(windlist, genlist, left_on='GeneratorName', right_on='Name', how='inner')

In [24]:
columns_to_drop = ['GeneratorKey','Type','CommitmentShapeID','CommitmentMultiplier','DispatchShapeID','DispatchShapeName',
                  'DispatchMultiplier','CommitmentVariation(%)','CommitmentPdf','DispatchVariation(%)','DispatchPdf',
                   'Spillage','IfEnableMultiBlock','ModelType','IfUseInvertLoadingRatio','InvertLoadingRatio','TargetCapacityFactor','Name']
windlist = windlist.drop(columns=columns_to_drop)

In [25]:
windlist.to_csv('Processed Data/Wind.csv')

In [26]:
windlist.iloc[0]

GeneratorName                         100B-Ekola Flats WD 1
CommitmentShapeName    WT_Ekola Flats Wind0WT250.9 A-1_[18]
Capacity(MW)                                          120.4
SubType                                          WT-Onshore
Area Name                                              PAWY
InitialDispatch(MW)                                    18.0
Zone                                                      5
Name: 0, dtype: object

# Wind  profile

In [27]:
windshapelist = pd.read_csv('Processed Data/Wind.csv', usecols=['GeneratorName','CommitmentShapeName','Capacity(MW)'])
windcruve = pd.read_csv('Public Data/Hourly Profiles in CSV Format/Hourly Profile_Wind.csv',header=0, skiprows=list(range(8786, 8796)))

C:\Users\wenmi\AppData\Local\Temp\ipykernel_8300\2173675073.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  windcruve = pd.read_csv('Public Data/Hourly Profiles in CSV Format/Hourly Profile_Wind.csv',header=0, skiprows=list(range(8786, 8796)))


In [28]:
windcruve = windcruve.drop(index=0)

In [29]:
windcruve.columns = [re.sub(r'\.dat:.*', '', col_name) for col_name in windcruve.columns]


In [30]:
windprofile = pd.DataFrame()
for index, row in windshapelist.iterrows():
    generator_name = row['GeneratorName']
    wind_profile_name = row['CommitmentShapeName']
    capacity = row['Capacity(MW)']
    wind_profile = windcruve[wind_profile_name]
    adjusted_profile = wind_profile * capacity
    windprofile[generator_name] = adjusted_profile

C:\Users\wenmi\AppData\Local\Temp\ipykernel_8300\1997807084.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  windprofile[generator_name] = adjusted_profile
C:\Users\wenmi\AppData\Local\Temp\ipykernel_8300\1997807084.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  windprofile[generator_name] = adjusted_profile
C:\Users\wenmi\AppData\Local\Temp\ipykernel_8300\1997807084.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Con

C:\Users\wenmi\AppData\Local\Temp\ipykernel_8300\1997807084.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  windprofile[generator_name] = adjusted_profile
C:\Users\wenmi\AppData\Local\Temp\ipykernel_8300\1997807084.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  windprofile[generator_name] = adjusted_profile
C:\Users\wenmi\AppData\Local\Temp\ipykernel_8300\1997807084.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Con

In [31]:
windprofile.to_csv('Processed Data/WindProfile.csv')

# Wind  map

In [32]:
windzone = pd.read_csv('Processed Data/Wind.csv', usecols=['GeneratorName','Zone'])

In [33]:
windmap = pd.get_dummies(windzone["Zone"])
windmap.to_csv('Processed Data/WindMap.csv')

# Solar processing

In [34]:
genlist = pd.read_csv('Public Data/GeneratorList.csv',header=2, encoding='ISO-8859-1', usecols=['Name','SubType','Area Name','InitialDispatch(MW)'])
hourlygenlist = pd.read_csv('Public Data/HourlyResource_General.csv',header=2, encoding='ISO-8859-1')

In [35]:
genlist['Zone'] = genlist['Area Name'].map(region.set_index('Area Name')['Zone'])
solarlist = hourlygenlist[hourlygenlist['Type'] == 'Solar']
solarlist = pd.merge(solarlist, genlist, left_on='GeneratorName', right_on='Name', how='inner')

In [36]:
columns_to_drop = ['GeneratorKey','Type','CommitmentShapeID','CommitmentMultiplier','DispatchShapeID','DispatchShapeName',
                  'DispatchMultiplier','CommitmentVariation(%)','CommitmentPdf','DispatchVariation(%)','DispatchPdf',
                   'Spillage','IfEnableMultiBlock','ModelType','IfUseInvertLoadingRatio','InvertLoadingRatio','TargetCapacityFactor','Name']
solarlist = solarlist.drop(columns=columns_to_drop)

In [37]:
solarlist.to_csv('Processed Data/Solar.csv')

# Solar profile


In [38]:
solarshapelist = pd.read_csv('Processed Data/Solar.csv', usecols=['GeneratorName','CommitmentShapeName','Capacity(MW)'])
solarcruve1 = pd.read_csv('Public Data/Hourly Profiles in CSV Format/Hourly Profile_Solar1.csv',header=0, skiprows=list(range(8786, 8796)))
solarcruve2 = pd.read_csv('Public Data/Hourly Profiles in CSV Format/Hourly Profile_Solar2.csv',header=0, skiprows=list(range(8786, 8796)))
solarcruve3 = pd.read_csv('Public Data/Hourly Profiles in CSV Format/Hourly Profile_BTM Solar_DPV.csv',header=0, skiprows=list(range(8786, 8796)))

C:\Users\wenmi\AppData\Local\Temp\ipykernel_8300\3365180844.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  solarcruve1 = pd.read_csv('Public Data/Hourly Profiles in CSV Format/Hourly Profile_Solar1.csv',header=0, skiprows=list(range(8786, 8796)))
C:\Users\wenmi\AppData\Local\Temp\ipykernel_8300\3365180844.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  solarcruve2 = pd.read_csv('Public Data/Hourly Profiles in CSV Format/Hourly Profile_Solar2.csv',header=0, skiprows=list(range(8786, 8796)))
C:\Users\wenmi\AppData\Local\Temp\ipykernel_8300\3365180844.py:4: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  solarcruve3 = pd.read_csv('Public Data/Hourly Profiles in CSV Format/Hourly Profile_BTM Solar_DPV.csv',header=0, skiprows=list(range(8786, 8796)))


In [39]:
solarcruve1 = solarcruve1.drop(index=0)
solarcruve2 = solarcruve2.drop(index=0)
solarcruve3 = solarcruve3.drop(index=0)

In [40]:
solarcruve1.columns = [re.sub(r'\.DAT: 2032$|\.dat: 2032$', '', col_name) for col_name in solarcruve1.columns]
solarcruve2.columns = [re.sub(r'\.DAT: 2032$|\.dat: 2032$', '', col_name) for col_name in solarcruve2.columns]
solarcruve3.columns = [re.sub(r'\.DAT: 2032$|\.dat: 2032$', '', col_name) for col_name in solarcruve3.columns]


solarcruve = pd.merge(solarcruve1, solarcruve2, on='Index')
solarcruve = pd.merge(solarcruve, solarcruve3, on='Index')



In [41]:
solarprofile = pd.DataFrame()
for index, row in solarshapelist.iterrows():
    generator_name = row['GeneratorName']
    solar_profile_name = row['CommitmentShapeName']
    capacity = row['Capacity(MW)']
    solar_profile = solarcruve[solar_profile_name]
    adjusted_profile = solar_profile * capacity
    solarprofile[generator_name] = adjusted_profile

C:\Users\wenmi\AppData\Local\Temp\ipykernel_8300\1595787671.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  solarprofile[generator_name] = adjusted_profile
C:\Users\wenmi\AppData\Local\Temp\ipykernel_8300\1595787671.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  solarprofile[generator_name] = adjusted_profile
C:\Users\wenmi\AppData\Local\Temp\ipykernel_8300\1595787671.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  C

C:\Users\wenmi\AppData\Local\Temp\ipykernel_8300\1595787671.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  solarprofile[generator_name] = adjusted_profile
C:\Users\wenmi\AppData\Local\Temp\ipykernel_8300\1595787671.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  solarprofile[generator_name] = adjusted_profile
C:\Users\wenmi\AppData\Local\Temp\ipykernel_8300\1595787671.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  C

C:\Users\wenmi\AppData\Local\Temp\ipykernel_8300\1595787671.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  solarprofile[generator_name] = adjusted_profile
C:\Users\wenmi\AppData\Local\Temp\ipykernel_8300\1595787671.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  solarprofile[generator_name] = adjusted_profile
C:\Users\wenmi\AppData\Local\Temp\ipykernel_8300\1595787671.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  C

C:\Users\wenmi\AppData\Local\Temp\ipykernel_8300\1595787671.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  solarprofile[generator_name] = adjusted_profile
C:\Users\wenmi\AppData\Local\Temp\ipykernel_8300\1595787671.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  solarprofile[generator_name] = adjusted_profile
C:\Users\wenmi\AppData\Local\Temp\ipykernel_8300\1595787671.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  C

C:\Users\wenmi\AppData\Local\Temp\ipykernel_8300\1595787671.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  solarprofile[generator_name] = adjusted_profile
C:\Users\wenmi\AppData\Local\Temp\ipykernel_8300\1595787671.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  solarprofile[generator_name] = adjusted_profile
C:\Users\wenmi\AppData\Local\Temp\ipykernel_8300\1595787671.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  C

In [42]:
solarprofile.to_csv('Processed Data/SolarProfile.csv')

# Solar  map

In [43]:
solarzone = pd.read_csv('Processed Data/Solar.csv', usecols=['GeneratorName','Zone'])
solarmap = pd.get_dummies(solarzone["Zone"])
solarmap.to_csv('Processed Data/SolarMap.csv')

# Hydro  processing

In [44]:
genlist = pd.read_csv('Public Data/GeneratorList.csv',header=2, encoding='ISO-8859-1', usecols=['Name','SubType','Area Name','MinCap(MW)','MaxCap(MW)','InitialDispatch(MW)'])
genlist['Zone'] = genlist['Area Name'].map(region.set_index('Area Name')['Zone'])
hydrolist = genlist.loc[genlist['SubType'].isin(['Hydro', 'HydroRPS'])]
hydrolist.to_csv('Processed Data/Hydro.csv')

In [45]:
hydrocruve = pd.read_csv('Public Data/Hourly Profiles in CSV Format/Hourly Profile_Hydro by Area.csv',header=0, skiprows=list(range(8786, 8796)))
hydrocruve = hydrocruve.drop(index=0)
hydrocruve.columns = [re.sub('HY_(.*?)_2018.dat: 2032', r'\1', col_name) for col_name in hydrocruve.columns]


In [46]:
hydroprofile = pd.DataFrame()
for index, row in hydrolist.iterrows():
    generator_name = row['Name']
    hydro_profile_name = row['Area Name']
    capacity = row['MaxCap(MW)']
    if hydro_profile_name == 'SRP':
        hydro_profile = hydrocruve['PNM']
    elif hydro_profile_name in {'CISC','CISD'}:
        hydro_profile = hydrocruve['CISC_CISD']
    elif hydro_profile_name in {'CIPV','CIPB'}:
        hydro_profile = hydrocruve['CIPV_CIPB']
    elif hydro_profile_name in {'CHPD','DOPD'}:
        hydro_profile = hydrocruve['GCPD']
    elif hydro_profile_name in {'PSCO','IPMV','IPTV','IPFE'}:
        hydro_profile = hydrocruve['PSCO_IPMV_IPTV_IPFE']
    elif hydro_profile_name == 'WAUW':
        hydro_profile = hydrocruve['WACM']
    elif hydro_profile_name in {'WALC','SPPC'}:
        hydro_profile = hydrocruve['PAUT']
    elif hydro_profile_name == 'TH_Mead':
        hydro_profile = hydrocruve['NEVP']
    else: 
        hydro_profile = hydrocruve[hydro_profile_name]
    adjusted_profile = hydro_profile * capacity
    hydroprofile[generator_name] = adjusted_profile

C:\Users\wenmi\AppData\Local\Temp\ipykernel_8300\1577005624.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  hydroprofile[generator_name] = adjusted_profile
C:\Users\wenmi\AppData\Local\Temp\ipykernel_8300\1577005624.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  hydroprofile[generator_name] = adjusted_profile
C:\Users\wenmi\AppData\Local\Temp\ipykernel_8300\1577005624.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.

C:\Users\wenmi\AppData\Local\Temp\ipykernel_8300\1577005624.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  hydroprofile[generator_name] = adjusted_profile
C:\Users\wenmi\AppData\Local\Temp\ipykernel_8300\1577005624.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  hydroprofile[generator_name] = adjusted_profile
C:\Users\wenmi\AppData\Local\Temp\ipykernel_8300\1577005624.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.

C:\Users\wenmi\AppData\Local\Temp\ipykernel_8300\1577005624.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  hydroprofile[generator_name] = adjusted_profile
C:\Users\wenmi\AppData\Local\Temp\ipykernel_8300\1577005624.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  hydroprofile[generator_name] = adjusted_profile
C:\Users\wenmi\AppData\Local\Temp\ipykernel_8300\1577005624.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.

In [47]:
hydroprofile.to_csv('Processed Data/HydroProfile.csv')

# Hydro  map


In [48]:
hydrozone = pd.read_csv('Processed Data/Hydro.csv', usecols=['Name','Zone'])
hydromap = pd.get_dummies(hydrozone["Zone"])
hydromap.to_csv('Processed Data/HydroMap.csv')

# Storage  processing

In [2]:
genlist = pd.read_csv('Public Data/GeneratorList.csv',header=2, encoding='ISO-8859-1', usecols=['Name','SubType','Area Name','MinCap(MW)','MaxCap(MW)','InitialDispatch(MW)'])
genlist['Zone'] = genlist['Area Name'].map(region.set_index('Area Name')['Zone'])

NameError: name 'region' is not defined

In [50]:
storagelist = genlist.loc[genlist['SubType'].isin(['PS-Hydro', 'PS-HydroRPS','Battery Storage'])]
storagelist['Efficiency'] = np.where(storagelist['SubType'].isin(['PS-Hydro', 'PS-HydroRPS']), 0.80, 
                            np.where(storagelist['SubType'] == 'Battery Storage', 0.90, np.nan))
storagelist['Duration(h)'] = np.where(storagelist['SubType'].isin(['PS-Hydro', 'PS-HydroRPS']), 12, 
                            np.where(storagelist['SubType'] == 'Battery Storage', 2, np.nan))
storagelist['MaxCap(MWh)'] = storagelist['MaxCap(MW)'] * storagelist['Duration(h)']


C:\Users\wenmi\AppData\Local\Temp\ipykernel_8300\549637819.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  storagelist['Efficiency'] = np.where(storagelist['SubType'].isin(['PS-Hydro', 'PS-HydroRPS']), 0.80,
C:\Users\wenmi\AppData\Local\Temp\ipykernel_8300\549637819.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  storagelist['Duration(h)'] = np.where(storagelist['SubType'].isin(['PS-Hydro', 'PS-HydroRPS']), 12,
C:\Users\wenmi\AppData\Local\Temp\ipykernel_8300\549637819.py:6: SettingWithCopyWarning: 

In [51]:
storagelist.to_csv('Processed Data/Storage.csv')

# Storage  map


In [6]:
storagezone = pd.read_csv('Processed Data/Storage.csv', usecols=['Name','Zone'])
storagemap = pd.get_dummies(storagezone["Zone"])
storagemap.to_csv('Processed Data/StorageMap.csv')